<a href="https://colab.research.google.com/github/UrukHan/NeuroNetStockQuotes/blob/master/AlphaVintageApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install alpha_vantage

     |████████████████████████████████| 1.2MB 4.1MB/s 
     |████████████████████████████████| 153kB 13.0MB/s 
     |████████████████████████████████| 266kB 17.3MB/s 
  Created wheel for alpha-vantage: filename=alpha_vantage-2.2.0-cp36-none-any.whl size=25706 sha256=630e682063c705bf545c19b5fa210dc235ad665736120baf691a0980266259cf
  Stored in directory: /root/.cache/pip/wheels/3a/fa/05/80b0afcbd2a4de8bd2a0c5cd77084e3d3a9cc12c48be9b7f84
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=28e379b43039278db571d3b86d14f60ae1ecd4e26a1e46ff818b752168c73527
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built alpha-vantage idna-ssl


In [2]:
import requests
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from datetime import timedelta
from google.colab import files
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.timeseries import TimeSeries
from datetime import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Очистить журналы от предыдущих вызовов
!rm -rf ./logs/ 
!ls

sample_data  stockQuotes-5m.csv


In [4]:
# Ввод ключа (IQLCQLXYBPW3LH7T)
key = '8C5RUT8LNE2P0F4O' #input('Введите ключ доступа   -   ') 

In [18]:
ts = TimeSeries(key = key)
# listCompany = ['MSFT', 'CAT', 'BA', 'GS', 'HD'] 
with open('CompaniesList.txt') as f:
  listTitleCompany = f.read()
  listCompany = listTitleCompany.split(" ")

In [19]:
# Скачивание исторических данных котировок акций
ts = TimeSeries(key = key, output_format = 'pandas')
# Счетчик для задержки между запросами из за количественного их ограничения сервером за минуту
pauseDownload = 0        
for i in range (len(listCompany)):
  # Обработка ошибок по API запросам
  try:
    # Задаем задержку по количеству запросов в минуту
    if pauseDownload % 30 == 0 and pauseDownload != 0:        
      time.sleep(61)
    if pauseDownload == 0:
      # Запрос данных через API
      data, meta_data = ts.get_intraday(symbol = listCompany[i], interval = '5min', outputsize = 'full')  
      # Преоброзование таблиц в нужный нам вид 
      data = data[['1. open', '5. volume']] 
      data.columns = [listCompany[i] + '-P',listCompany[i] + '-V']
      if pauseDownload % 10 == 0 and pauseDownload != 0:
        print(listCompany[i], data.shape[0], end = '\n')
      else:
        print(listCompany[i], data.shape[0], end = '   ')
    else:
      dataConcat, meta_data = ts.get_intraday(symbol = listCompany[i],interval = '5min', outputsize = 'full')
      dataConcat = dataConcat[['1. open', '5. volume']]
      dataConcat.columns = [listCompany[i] + '-P',listCompany[i] + '-V']
      if pauseDownload % 10 == 0 and pauseDownload != 0:
        print(listCompany[i], dataConcat.shape, end = '\n')
      else:
        print(listCompany[i], dataConcat.shape, end = '   ')
      # Объединение таблиц
      data = pd.concat([data, dataConcat], axis = 1)
  except ValueError:
    print(listCompany[i], "Что-то пошло не так")
  pauseDownload += 1
print(data)

MMM 1820   ABT (2136, 2)   ABBV (2031, 2)   ACN (1642, 2)   ATVI (2040, 2)   ADBE (2109, 2)   ADT (1757, 2)   AAP (1642, 2)   AET Что-то пошло не так
AMG (1570, 2)   AFL (1670, 2)
A (1642, 2)   APD (1602, 2)   ARG Что-то пошло не так
AKAM (1701, 2)   AA (1951, 2)   ALXN (1657, 2)   ALLE (1589, 2)   AGN Что-то пошло не так
ADS (1649, 2)   ALL (1641, 2)
GOOG (1864, 2)   MO (2088, 2)   AMZN (2605, 2)   AEP (1653, 2)   AEE (1602, 2)   AXP (1913, 2)   AIG (1838, 2)   AMT (1622, 2)   AMP (1605, 2)   ABC (1616, 2)
AME (1592, 2)   AMGN (1837, 2)   APH (1598, 2)   APC Что-то пошло не так
ADI (1734, 2)   AAL (3704, 2)   ANTM (1607, 2)   AON (1617, 2)   APA (2250, 2)   AIV (1596, 2)
AAPL (3740, 2)   AMAT (2210, 2)   ADM (1648, 2)   AIZ (1569, 2)   T (2954, 2)   ADSK (1719, 2)   ADP (1673, 2)   AN (1621, 2)   AZO (1457, 2)   AVGO (1980, 2)
AVB (1599, 2)   AVY (1593, 2)   BHI Что-то пошло не так
BLL (1620, 2)   BAC (3322, 2)   BXTL Что-то пошло не так
BAX (1605, 2)   BBT Что-то пошло не так
BDX (16

In [20]:
# Скачивание исторических данных курсов валют
fe = ForeignExchange(key = key, output_format = 'pandas')
listF = ['EUR', 'BTC', 'CHF', 'JPY', 'CNY', 'GBP', 'RUB']
for i in listF:
  # Обработка ошибок по API запросам
  try:     
    if i == listF[0]:
      # Запрос данных через API
      dataF, meta_dataF = fe.get_currency_exchange_intraday(from_symbol = i, to_symbol = 'USD', interval = '5min', outputsize = 'full') 
      # Преоброзование таблиц в нужный нам вид 
      dataF = pd.DataFrame(dataF['1. open'])
      dataF.columns = [i]
    else:
      dataConcatF, meta_dataF = fe.get_currency_exchange_intraday(from_symbol = i, interval = '5min', to_symbol = 'USD', outputsize = 'full') 
      dataConcatF = pd.DataFrame(dataConcatF['1. open'])
      dataConcatF.columns = [i]
      dataF = pd. concat([dataF, dataConcatF], axis = 1)
    print(i, dataF.shape[0])
  except ValueError:
    print(i, "Что-то пошло не так", end = "\n")
print(dataF)

EUR 974
BTC Что-то пошло не так
CHF 974
JPY 974
CNY 1799
GBP 1799
RUB 1799
                        EUR     CHF     JPY     CNY     GBP     RUB
date                                                               
2020-09-03 07:20:00     NaN     NaN     NaN  0.1463     NaN     NaN
2020-09-03 07:25:00     NaN     NaN     NaN  0.1463     NaN     NaN
2020-09-03 07:30:00     NaN     NaN     NaN  0.1463     NaN     NaN
2020-09-03 07:35:00     NaN     NaN     NaN  0.1463     NaN     NaN
2020-09-03 07:40:00     NaN     NaN     NaN  0.1463     NaN     NaN
...                     ...     ...     ...     ...     ...     ...
2020-09-15 14:55:00  1.1847  1.1008  0.0095  0.1474  1.2858  0.0133
2020-09-15 15:00:00  1.1855  1.1015  0.0095  0.1474  1.2862  0.0133
2020-09-15 15:05:00  1.1850  1.1010  0.0095  0.1474  1.2854  0.0133
2020-09-15 15:10:00  1.1845  1.1005  0.0095  0.1474  1.2845  0.0133
2020-09-15 15:15:00  1.1851  1.1007  0.0095  0.1474  1.2847     NaN

[1799 rows x 6 columns]


In [21]:
allData = pd. concat([data, dataF], axis = 1)
allData.shape

(4550, 838)

In [22]:
allData.iloc[400:420, 0:20]

,MMM-P,MMM-V,ABT-P,ABT-V,ABBV-P,ABBV-V,ACN-P,ACN-V,ATVI-P,ATVI-V,ADBE-P,ADBE-V,ADT-P,ADT-V,AAP-P,AAP-V,AMG-P,AMG-V,AFL-P,AFL-V
date,,,,,,,,,,,,,,,,,,,,
2020-08-19 05:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-19 05:30:00,NaN,NaN,100.80,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-19 05:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-19 05:40:00,NaN,NaN,100.78,510.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-19 05:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-19 05:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-19 05:55:00,NaN,NaN,NaN,NaN,96.49,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-19 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-19 06:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Определение времени для выбрки времени
dayStart = datetime.now() - timedelta(days = 4)
dayStart = dayStart.replace(hour=4, minute=0, second=0, microsecond=0)
dayEnd = datetime.now() - timedelta(days = 1)
dayEnd = dayEnd.replace(hour=20, minute=0, second=0, microsecond=0)
print(dayStart)
print(dayEnd)

2020-09-11 04:00:00
2020-09-14 20:00:00


In [24]:
# Изменяем формат даты в индексе и вырезаем по времени нужный отрезок
#allData.set_index(allData.index.strftime("%H:%M %d-%m-%Y"), inplace=True)
allDataSlice = allData.loc[dayStart:dayEnd]
allDataSlice.shape

(480, 838)

In [41]:
allDataSlice.iloc[160:165, 824:844]

,XYL-P,XYL-V,YUM-P,YUM-V,ZION-P,ZION-V,ZTS-P,ZTS-V,EUR,CHF,JPY,CNY,GBP,RUB
date,,,,,,,,,,,,,,
2020-09-11 17:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1831,1.0991,0.0094,NaN,1.2781,0.0133
2020-09-11 17:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1836,1.0994,0.0094,NaN,1.2790,0.0133
2020-09-11 17:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1835,1.0993,0.0094,NaN,1.2788,0.0133
2020-09-11 17:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1831,1.0990,0.0094,NaN,1.2778,0.0133
2020-09-11 17:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1830,1.0992,0.0094,NaN,1.2780,0.0133


In [42]:
allDataSliceTime = allDataSlice.query('index.dt.hour > 3 and index.dt.hour < 20')

In [43]:
allDataSliceTime.shape

(384, 838)

In [44]:
print(allDataSliceTime)

                     MMM-P  MMM-V  ABT-P  ABT-V  ...     JPY     CNY     GBP     RUB
date                                             ...                                
2020-09-11 04:00:00    NaN    NaN    NaN    NaN  ...  0.0094  0.1462  1.2825  0.0132
2020-09-11 04:05:00    NaN    NaN    NaN    NaN  ...  0.0094  0.1462  1.2828  0.0133
2020-09-11 04:10:00    NaN    NaN    NaN    NaN  ...  0.0094  0.1462  1.2828  0.0133
2020-09-11 04:15:00    NaN    NaN    NaN    NaN  ...  0.0094  0.1462  1.2829  0.0132
2020-09-11 04:20:00    NaN    NaN    NaN    NaN  ...  0.0094  0.1462  1.2829  0.0132
...                    ...    ...    ...    ...  ...     ...     ...     ...     ...
2020-09-14 19:35:00    NaN    NaN    NaN    NaN  ...  0.0095     NaN  1.2863  0.0132
2020-09-14 19:40:00    NaN    NaN    NaN    NaN  ...  0.0095     NaN  1.2862  0.0132
2020-09-14 19:45:00    NaN    NaN    NaN    NaN  ...  0.0095     NaN  1.2861  0.0132
2020-09-14 19:50:00    NaN    NaN    NaN    NaN  ...  0.0095     

In [79]:
# Загрузка данных
allDataOld = pd.read_csv('/content/stockQuotes-5m.csv')
# allDataOld = allDataOld.iloc[:300]
allDataOld.index = pd.DatetimeIndex(allDataOld['date'])
allDataOld.drop(['date'], axis=1, inplace=True)
allDataOld.iloc[-5:, 0:20]

,MMM-P,MMM-V,ABT-P,ABT-V,ABBV-P,ABBV-V,ACN-P,ACN-V,ATVI-P,ATVI-V,ADBE-P,ADBE-V,ADT-P,ADT-V,AAP-P,AAP-V,AMG-P,AMG-V,AFL-P,AFL-V
date,,,,,,,,,,,,,,,,,,,,
2020-09-14 12:35:00,168.425,10530.0,106.3685,24881.0,91.19,39535.0,240.56,5446.0,78.890,25493.0,485.99,38927.0,10.865,6740.0,157.705,8026.0,66.780,8546.0,37.240,18129.0
2020-09-14 12:40:00,168.450,12797.0,106.3300,18684.0,91.12,26788.0,240.27,5064.0,78.765,42382.0,484.00,27864.0,10.845,10570.0,158.080,1438.0,66.730,2001.0,37.295,26302.0
2020-09-14 12:45:00,168.575,13949.0,106.3100,17853.0,91.10,49883.0,240.14,7715.0,78.780,20959.0,483.26,46742.0,10.845,18769.0,158.185,2414.0,66.950,1029.0,37.355,20814.0
2020-09-14 12:50:00,168.600,7548.0,106.2400,22424.0,91.09,23376.0,240.21,4807.0,78.920,20934.0,485.04,20153.0,10.840,5944.0,158.150,6803.0,66.945,2949.0,37.350,20839.0
2020-09-14 12:55:00,168.590,15455.0,106.2300,16589.0,91.05,20176.0,239.93,8363.0,78.870,32015.0,484.50,22857.0,10.830,7261.0,157.930,3740.0,66.985,2156.0,37.350,39898.0


In [80]:
# Нахождение последнего индекса таблицы (последняя запись даты)
daySliceAdd = allDataOld.iloc[-1].name
daySliceAdd = daySliceAdd + timedelta(minutes = 5)

In [82]:
addData = allDataSliceTime.loc[daySliceAdd:]
print(addData)

                      MMM-P    MMM-V   ABT-P  ...     CNY     GBP     RUB
date                                          ...                        
2020-09-14 13:00:00  168.76  21955.0  106.17  ...  0.1467  1.2867  0.0132
2020-09-14 13:05:00  168.84  39260.0  106.25  ...  0.1467  1.2874  0.0132
2020-09-14 13:10:00  169.06  26661.0  106.17  ...  0.1467  1.2891  0.0132
2020-09-14 13:15:00  168.71  18444.0  106.08  ...  0.1468  1.2902  0.0132
2020-09-14 13:20:00  169.03  20769.0  106.05  ...  0.1468  1.2893  0.0132
...                     ...      ...     ...  ...     ...     ...     ...
2020-09-14 19:35:00     NaN      NaN     NaN  ...     NaN  1.2863  0.0132
2020-09-14 19:40:00     NaN      NaN     NaN  ...     NaN  1.2862  0.0132
2020-09-14 19:45:00     NaN      NaN     NaN  ...     NaN  1.2861  0.0132
2020-09-14 19:50:00     NaN      NaN     NaN  ...     NaN  1.2860  0.0132
2020-09-14 19:55:00     NaN      NaN     NaN  ...     NaN  1.2859  0.0132

[84 rows x 838 columns]


In [85]:
allDataSliceTime = pd.concat([allDataOld, addData], axis = 0)
print(allDataSliceTime)

                     MMM-P  MMM-V  ABT-P  ABT-V  ...     JPY     CNY     GBP     RUB
date                                             ...                                
2020-09-11 04:00:00    NaN    NaN    NaN    NaN  ...  0.0094  0.1462  1.2825  0.0132
2020-09-11 04:05:00    NaN    NaN    NaN    NaN  ...  0.0094  0.1462  1.2828  0.0133
2020-09-11 04:10:00    NaN    NaN    NaN    NaN  ...  0.0094  0.1462  1.2828  0.0133
2020-09-11 04:15:00    NaN    NaN    NaN    NaN  ...  0.0094  0.1462  1.2829  0.0132
2020-09-11 04:20:00    NaN    NaN    NaN    NaN  ...  0.0094  0.1462  1.2829  0.0132
...                    ...    ...    ...    ...  ...     ...     ...     ...     ...
2020-09-14 19:35:00    NaN    NaN    NaN    NaN  ...  0.0095     NaN  1.2863  0.0132
2020-09-14 19:40:00    NaN    NaN    NaN    NaN  ...  0.0095     NaN  1.2862  0.0132
2020-09-14 19:45:00    NaN    NaN    NaN    NaN  ...  0.0095     NaN  1.2861  0.0132
2020-09-14 19:50:00    NaN    NaN    NaN    NaN  ...  0.0095     

In [87]:
# Сохранение и скачивание данных
allDataSliceTime.to_csv('stockQuotes-5m.csv')
files.download('stockQuotes-5m.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>